In [ ]:
!pip install gensim
!pip install nltk

In [ ]:
from gensim.models import KeyedVectors

# Pobranie modelu
model_url = "https://huggingface.co/clarin-pl/word2vec-kgr10/resolve/main/skipgram.v300.m8.ns.mwe.w2v.gensim"
model_path = "word2vec-kgr10.gensim"

# Pobranie pliku
import urllib.request
urllib.request.urlretrieve(model_url, model_path)

# Download the vectors file separately
vectors_url = "https://huggingface.co/clarin-pl/word2vec-kgr10/resolve/main/skipgram.v300.m8.ns.mwe.w2v.gensim.vectors.npy" # URL of the vectors file
vectors_path = "word2vec-kgr10.gensim.vectors.npy" # Local path for the vectors file
urllib.request.urlretrieve(vectors_url, vectors_path)

# Załadowanie modelu
model = KeyedVectors.load(model_path)

In [ ]:
import pandas as pd

reviews = pd.read_csv("filmweb_jednolity_sentyment.csv")
reviews_with_idioms = pd.read_csv('filmweb_i_idiomy.csv')
print(reviews.head())

                                              review  sentiment
0  Nemo Nobody, bohater pierwszego od ponad dekad...          0
1  "Dziewczyna moich koszmarów" zaczyna się w mom...          0
2  Kultowy w pewnych kręgach horror Joe'ego D'Ama...          1
3  Motyw znany i często eksploatowany przez kino:...          1
4  Zwyczajna rozbita koreańska rodzina: rodzice r...          0


Tutaj trzeba dodać text preprocessing

In [ ]:
import re
import numpy as np
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

# Wczytanie listy polskich stop words z pliku
# with open('polish.stopwords.txt', 'r', encoding='utf-8') as file:
#     polish_stopwords = set(file.read().splitlines())

def preprocess_text(text):
    # Konwersja na małe litery
    text = text.lower()

    # Usunięcie znaków interpunkcyjnych
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenizacja
    words = text.split()

    # Usunięcie stop words
    # words = [word for word in words if word not in polish_stopwords]

    # Połączenie słów z powrotem w tekst
    return ' '.join(words)

# Zastosowanie przetwarzania tekstu dla kolumny 'review'
reviews['review_processed'] = reviews['review'].apply(preprocess_text)
reviews_with_idioms['review_processed'] = reviews_with_idioms['review'].apply(preprocess_text)

# Wyświetlenie przykładowych przetworzonych recenzji
# print("Przykładowe przetworzone recenzje:")
# print(reviews[['review', 'review_processed']].head(2))

# Aktualizacja funkcji wektoryzacji, aby używała przetworzonych recenzji
def vectorize_review(text, model):
    words = word_tokenize(text.lower())
    words = [word for word in words if word in model.key_to_index]

    if words:
        return np.mean(model[words], axis=0)  # Średnia wektorów słów
    else:
        return np.zeros(model.vector_size)  # Zerowy wektor, jeśli brak znanych słów

# Aktualizacja wektorów dla przetworzonych recenzji
reviews['vector'] = reviews['review_processed'].apply(lambda x: vectorize_review(x, model))
reviews_with_idioms['vector'] = reviews_with_idioms['review_processed'].apply(lambda x: vectorize_review(x, model))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
# import numpy as np
# import nltk
# nltk.download('punkt_tab')
# from nltk.tokenize import word_tokenize

# def vectorize_review(text, model):
#     words = word_tokenize(text.lower())
#     words = [word for word in words if word in model.key_to_index]

#     if words:
#         return np.mean(model[words], axis=0)  # Średnia wektorów słów
#     else:
#         return np.zeros(model.vector_size)  # Zerowy wektor, jeśli brak znanych słów

# # Tworzenie wektorów dla wszystkich recenzji
# reviews['vector'] = reviews['review_processed'].apply(lambda x: vectorize_review(x, model))


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Przygotowanie danych
X = np.vstack(reviews['vector'].values)
y = reviews['sentiment'].values
X2 = np.vstack(reviews_with_idioms['vector'].values)
y2 = reviews_with_idioms['sentiment'].values

# Podział na zbiory treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=None, random_state=12)

# Trening modelu kNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X2_train, y2_train)

# Predykcja i ocena
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

          -1       0.47      0.37      0.41       125
           0       0.61      0.68      0.64       371
           1       0.58      0.56      0.57       285

    accuracy                           0.59       781
   macro avg       0.56      0.54      0.54       781
weighted avg       0.58      0.59      0.58       781

